In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive

/content/drive


In [5]:
%cd Shareddrives/blisssinghal-dataset/

/content/drive/Shareddrives/blisssinghal-dataset


In [6]:
%pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.7 MB/s eta 0:00:00


In [7]:
import glob
import numpy as np
import pandas as pd
import mne
import os
import matplotlib.pyplot as plt
import sklearn 
import keras
import pickle
import pickletools
import sklearn.neighbors
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler

In [8]:
from keras.layers import (LSTM, Dense, RNN, Flatten, Dropout, SimpleRNN, Conv1D)
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers


In [7]:
def flatten(names):
    new_array = []
    for i in range(len(names)):
        for j in range(len(names[i])):
            new_array.append(names[i][j])
    return new_array

In [11]:
def save_epochs(epochs, file):
    with open(file, 'wb') as file: 
        pickle.dump(epochs, file)

In [12]:
def load_epochs(file_name):
    with open(file_name, 'rb') as f:
        epochs = pickle.load(f)
    return epochs

In [13]:
epochs_array = load_epochs("epoch_files/epochs_with_ica.pkl")

In [11]:
def make_2d(array):
    dim1, dim2, dim3 = array.shape
    array = np.reshape(array, (dim1, dim2 * dim3))
    return array

In [12]:
epochs_2d = make_2d(epochs_array)

In [13]:
def transfer_labels2(time, epoch_labels, padding_time = 0):
  #note: time is in seconds
  shift_epochs = int(time / 5)
  padd_epochs = int(padding_time / 5)
  for index in range(len(epoch_labels)):
    #doesnt shift epoch labels if the seizure happens too early 
    if epoch_labels[index] == 1 and index - shift_epochs >= 0:
      epoch_labels[index - shift_epochs] = 1
      #padding before
      for i in range(padd_epochs):
        if (index - (shift_epochs + padd_epochs) >= 0):
          epoch_labels[index - (shift_epochs + padd_epochs)] = 1
      epoch_labels[index] = 0
  return epoch_labels

In [14]:
def get_all_epoch_labels(label_folder, time_before, padding_time):
  labels = []
  for i in range(24):
    print(i)
    if i == 0 or i == 3 or i == 12 or i == 13 or i == 17:
      continue
    if i < 10:
      f = "chb0" + str(i) + "_labels.pkl"
    else:
      f = "chb" + str(i) + "_labels.pkl"
    
    file_labels = load_epochs(f"{label_folder}/{f}") 
    file_labels = transfer_labels2(time_before, file_labels, padding_time)
    labels.append(file_labels)
    #flattening the labels array
  labels = flatten(labels)
  return labels


In [15]:
labels = get_all_epoch_labels("label_files", (20 * 60), 40 * 60)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [14]:
labels = load_epochs("label_files/20min_transfer_40minpadd.pkl")

In [15]:
labels = np.asarray(labels)

In [18]:
def lstm(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2, dim3 = x_train.shape
  model = keras.Sequential()
  
  model.add(LSTM(128, return_sequences = True, activation = "tanh", input_shape = (dim2, dim3)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  model.add(LSTM(16, activation = 'tanh', input_shape = (128,)))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [19]:
lstm(epochs_array[:200000], labels[:200000], epochs_array[200000:230000], labels[200000:230000], 30)

Epoch 1/30
6250/6250 [==============================] - 59s 8ms/step - loss: 0.6537 - binary_accuracy: 0.6411 - precision: 0.4286 - recall: 8.3598e-05 - false_positives: 8.0000 - false_negatives: 71766.0000 - true_positives: 6.0000 - true_negatives: 128220.0000 - val_loss: 0.6302 - val_binary_accuracy: 0.6781 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_positives: 0.0000e+00 - val_false_negatives: 9656.0000 - val_true_positives: 0.0000e+00 - val_true_negatives: 20344.0000
Epoch 2/30
6250/6250 [==============================] - 46s 7ms/step - loss: 0.6397 - binary_accuracy: 0.6560 - precision: 0.6298 - recall: 0.1006 - false_positives: 4246.0000 - false_negatives: 64550.0000 - true_positives: 7222.0000 - true_negatives: 123982.0000 - val_loss: 0.6091 - val_binary_accuracy: 0.6819 - val_precision: 0.7591 - val_recall: 0.0173 - val_false_positives: 53.0000 - val_false_negatives: 9489.0000 - val_true_positives: 167.0000 - val_true_negatives: 20291.0000
Epoch 3/30
6250/6

In [20]:
def knn(x_train, y_train, x_test, y_test):
  model = sklearn.neighbors.KNeighborsClassifier()
  model.fit(x_train, y_train)
  y_preds = model.predict(x_test)
  score = sklearn.metrics.accuracy_score(y_test, y_preds)
  return score

In [21]:
score = knn(epochs_2d[:200000], labels[:200000], epochs_2d[200000:230000], labels[200000:230000])

In [22]:
print(score)

0.6695666666666666


In [23]:
def logistic_regression(x_train, y_train, x_test, y_test):
  model = sklearn.linear_model.LogisticRegression()
  model.fit(x_train, y_train)
  y_preds = model.predict(x_test)
  score = sklearn.metrics.accuracy_score(y_test, y_preds)
  return score

In [24]:
score = logistic_regression(epochs_2d[:200000], labels[:200000], epochs_2d[200000:230000], labels[200000:230000])

In [25]:
print(score)

0.6781333333333334


In [16]:
def rnn(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2, dim3 = x_train.shape
  model = keras.Sequential()
  
  model.add(SimpleRNN(128, return_sequences = True, activation = "tanh", input_shape = (dim2, dim3)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  model.add(SimpleRNN(16, activation = 'tanh', input_shape = (128,)))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [17]:
rnn(epochs_array[:200000], labels[:200000], epochs_array[200000:230000], labels[200000:230000], 30)

Epoch 1/30
6250/6250 [==============================] - 115s 17ms/step - loss: 0.6385 - binary_accuracy: 0.6551 - precision: 0.5927 - recall: 0.1248 - false_positives: 6153.0000 - false_negatives: 62817.0000 - true_positives: 8955.0000 - true_negatives: 122075.0000 - val_loss: 0.5714 - val_binary_accuracy: 0.6827 - val_precision: 0.7134 - val_recall: 0.0237 - val_false_positives: 92.0000 - val_false_negatives: 9427.0000 - val_true_positives: 229.0000 - val_true_negatives: 20252.0000
Epoch 2/30
6250/6250 [==============================] - 103s 17ms/step - loss: 0.6092 - binary_accuracy: 0.6773 - precision: 0.6090 - recall: 0.2819 - false_positives: 12991.0000 - false_negatives: 51539.0000 - true_positives: 20233.0000 - true_negatives: 115237.0000 - val_loss: 0.5532 - val_binary_accuracy: 0.7477 - val_precision: 0.8115 - val_recall: 0.2817 - val_false_positives: 632.0000 - val_false_negatives: 6936.0000 - val_true_positives: 2720.0000 - val_true_negatives: 19712.0000
Epoch 3/30
6250/6250

In [18]:
def cnn(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2, dim3 = x_train.shape
  print(dim3)
  model = keras.Sequential()
  
  model.add(Conv1D(2, 10, activation = "relu", input_shape = (dim2, dim3)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  #model.add(Conv1D(16, 50, activation = 'relu'))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [19]:
cnn(epochs_array[:200000], labels[:200000], epochs_array[200000:230000], labels[200000:230000], 30)

1280
Epoch 1/30
6250/6250 [==============================] - 43s 6ms/step - loss: 0.6537 - binary_accuracy: 0.6411 - precision_1: 0.4286 - recall_1: 1.6720e-04 - false_positives_1: 16.0000 - false_negatives_1: 71760.0000 - true_positives_1: 12.0000 - true_negatives_1: 128212.0000 - val_loss: 0.6298 - val_binary_accuracy: 0.6781 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_false_positives_1: 0.0000e+00 - val_false_negatives_1: 9656.0000 - val_true_positives_1: 0.0000e+00 - val_true_negatives_1: 20344.0000
Epoch 2/30
6250/6250 [==============================] - 33s 5ms/step - loss: 0.6529 - binary_accuracy: 0.6411 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - false_positives_1: 0.0000e+00 - false_negatives_1: 71772.0000 - true_positives_1: 0.0000e+00 - true_negatives_1: 128228.0000 - val_loss: 0.6315 - val_binary_accuracy: 0.6781 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_false_positives_1: 0.0000e+00 - val_false_negatives_1: 9656.0000 - val_tru